

importing library



In [9]:
pip install requests beautifulsoup4

In [10]:
pip install requests requests

In [69]:
# importing library

In [64]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [38]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string


In [39]:
# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

In [40]:
# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

In [41]:
# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

In [42]:
# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [46]:
if __name__ == '__main__':

# add your user agent
 HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

In [47]:
# The webpage URL
URL = "https://www.amazon.in/s?rh=n%3A6612025031&fs=true&ref=lp_6612025031_sar"

In [48]:
 # HTTP Request
webpage = requests.get(URL, headers=HEADERS)

In [49]:
 # Soup Object containing all data
soup = BeautifulSoup(webpage.content, "html.parser")

In [50]:
# Fetch links as List of Tag Objects
links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

In [51]:
 # Store the links
links_list = []

In [57]:
# Loop for extracting links from Tag Objects
for link in links:
            links_list.append(link.get('href'))

d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

In [65]:
# Loop for extracting product details from each link
for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

# Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))


amazon_df = pd.DataFrame.from_dict(d)
amazon_df['title'].replace('', np.nan, inplace=True)
amazon_df = amazon_df.dropna(subset=['title'])
amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [68]:
amazon_df

,title,price,rating,reviews,availability
27,"Anker Power Bank, 10,000mAh Portable Charger (...",,4.5 out of 5 stars,"1,497 ratings",In Stock
31,DR VAKU 10000mAh Power Bank Clear Bolt Portabl...,,4.4 out of 5 stars,172 ratings,Not Available
60,"Anker Power Bank, 10,000mAh Portable Charger (...",,4.5 out of 5 stars,"1,497 ratings",In Stock
64,DR VAKU 10000mAh Power Bank Clear Bolt Portabl...,,4.4 out of 5 stars,172 ratings,Not Available


from matplotlib import pyplot as plt
import seaborn as sns
amazon_df.groupby('title').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
amazon_df.groupby('rating').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
amazon_df.groupby('reviews').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
amazon_df.groupby('availability').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['rating'].value_counts()
    for x_label, grp in amazon_df.groupby('title')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('title')
_ = plt.ylabel('rating')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['reviews'].value_counts()
    for x_label, grp in amazon_df.groupby('rating')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('rating')
_ = plt.ylabel('reviews')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['availability'].value_counts()
    for x_label, grp in amazon_df.groupby('reviews')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('reviews')
_ = plt.ylabel('availability')